In [1]:
import numpy as np
import pandas as pd
import csv
from scamp import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten,Softmax,Input
import keras.backend as K 

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Species.csv', sep=",",index_col = 0)
n = df.shape[0]
X = tf.Variable(np.zeros((2,n*2)),dtype='int32')  # any data tensor
for i in range(0,2*n,2):
    X[0,i].assign(df['Cantus'][i/2] - 43)
    X[1,i].assign(0)
    X[0,i+1].assign(df['Counter'][i/2] - 55)
    X[1,i+1].assign(1)
X

<tf.Variable 'Variable:0' shape=(2, 20000) dtype=int32, numpy=
array([[15, 12, 23, ..., 10, 13,  9],
       [ 0,  1,  0, ...,  1,  0,  1]])>

In [3]:
depth = 30
X = tf.concat([tf.one_hot(X[0,:], depth),tf.dtypes.cast(tf.reshape(X[1,:],[20000,1]),tf.float32)],1)
X

<tf.Tensor: shape=(20000, 31), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [4]:
def creat_model():
    
    rnn_units = 12
    n_notes = 30

    input_A = keras.layers.Input(shape=(20,31), name="input_A") # (15, 20, 31)
    input_B = keras.layers.Input(shape=(1,31), name="input_B") # (15, 1, 31)

    hidden1 = keras.layers.LSTM(rnn_units, return_sequences=True)(input_A) # (15,20,12)
    hidden2 = keras.layers.LSTM(rnn_units, return_sequences=True)(hidden1) #(15,20,12)
    
    output_RNN = keras.layers.Dense(n_notes, activation='softmax', name = 'output_RNN')(hidden2) #(15,20,30)

    e = keras.layers.Dense(1, activation='tanh')(hidden2) #(15,20,1)
    e = keras.layers.Reshape([-1])(e) #(15,20)

    alpha = keras.layers.Activation('softmax')(e) #(15,20)
    c = keras.layers.Permute([2, 1])(keras.layers.RepeatVector(rnn_units)(alpha)) #(15,20,12)
    c = keras.layers.Multiply()([hidden2, c]) #(15,20,12)
    c = keras.layers.Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c) #(15,12)

    output_A = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_A')(c) #(15,30) (0, cantus)

    aux = keras.layers.SimpleRNN(rnn_units)(input_B, initial_state=c) #(15,12)
    output_B = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B')(aux) #(15,30) (1, counter)

    model = keras.models.Model([input_A, input_B], [output_RNN, output_A, output_B])
    aux_model = keras.models.Model([input_A, input_B], [output_A, output_B])
    sub_model = keras.models.Model(input_A, output_A)
    att_model = keras.models.Model([input_A, input_B], alpha)
    
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy','categorical_crossentropy'],
              loss_weights=[0.2, 0.4, 0.4],optimizer="sgd")
    aux_model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'],
              optimizer="sgd")
    sub_model.compile(loss='categorical_crossentropy',
              optimizer="sgd")
    return model,aux_model,sub_model,att_model

In [5]:
model_0_1_,aux_model_0_1_, sub_model_0_1_,att_model_0_1_ = creat_model()
model_1_0_,aux_model_1_0_, sub_model_1_0_,att_model_1_0_ = creat_model()

In [6]:
model_0_1_.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_A (InputLayer)            [(None, 20, 31)]     0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 20, 12)       2112        input_A[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 20, 12)       1200        lstm[0][0]                       
__________________________________________________________________________________________________
dense (Dense)                   (None, 20, 1)        13          lstm_1[0][0]                     
_______________________________________________________________________________________

In [7]:
model_0_1_.load_weights('model_0_1_weight.h5')
model_1_0_.load_weights('model_1_0_weight.h5')

In [8]:
def generate_sequence(initial,seq_length):
    """
    Arguments:
    initial -- initial sequence of shape (1,20,31) (1 sample, sequence of length 20, vector fearure: one-hot(30);cantus(0),counter(1))
    seq_length -- # pairs of notes generated
    
    """
    ### START CODE HERE ###
    sequence = initial
    pred_0_1_0 = []
    pred_0_1_1 = []
    pred_1_0_1 = []
    pred_1_0_0 = []
    
    for i in range(seq_length):
        output_A_0_1_ = sub_model_0_1_.predict(initial).reshape([30,])
        pred_A_class_random_0_1_ = np.random.choice(30, 1, p=output_A_0_1_)[0]
        pred_0_1_0.append(pred_A_class_random_0_1_+43)
        
        output_A_1_0_ = sub_model_1_0_.predict(initial).reshape([30,])
        pred_A_class_random_1_0_ = np.random.choice(30, 1, p=output_A_1_0_)[0]
        pred_1_0_1.append(pred_A_class_random_1_0_+55)

        one_hot = tf.reshape(tf.one_hot(pred_A_class_random_0_1_, 30),[1,1,30])
        out_0_1_0 = tf.concat((one_hot,[[[0]]]),2)
        
        one_hot = tf.reshape(tf.one_hot(pred_A_class_random_1_0_, 30),[1,1,30])
        out_1_0_1 = tf.concat((one_hot,[[[1]]]),2)

        output_RNN, y_pred_A_0_1_, y_pred_B_0_1_ = model_0_1_.predict((initial,out_0_1_0))
        output_B_0_1_ = y_pred_B_0_1_.reshape([30,])
        pred_B_class_random_0_1_ = np.random.choice(30, 1, p=output_B_0_1_)[0]
        pred_0_1_1.append(pred_B_class_random_0_1_+55)
        
        output_RNN, y_pred_A_1_0_, y_pred_B_1_0_ = model_1_0_.predict((initial,out_1_0_1))
        output_B_1_0_ = y_pred_B_1_0_.reshape([30,])
        pred_B_class_random_1_0_ = np.random.choice(30, 1, p=output_B_1_0_)[0]
        pred_1_0_0.append(pred_B_class_random_1_0_+43)

        one_hot = tf.reshape(tf.one_hot(pred_B_class_random_0_1_, 30),[1,1,30])
        out_0_1_1 = tf.concat((one_hot,[[[1]]]),2)
        
        one_hot = tf.reshape(tf.one_hot(pred_B_class_random_1_0_, 30),[1,1,30])
        out_1_0_0 = tf.concat((one_hot,[[[0]]]),2)
        
        sequence = tf.concat((sequence,out_1_0_0),1)
        sequence = tf.concat((sequence,out_0_1_1),1)

        initial = sequence[:,-20:,:]
        
    return sequence,pred_0_1_0,pred_0_1_1,pred_1_0_1,pred_1_0_0

In [9]:
X_start_A = tf.reshape(X[9600:9620,:],[1,20,31])
seq_length = 10
sequence,pred_0_1_0,pred_0_1_1,pred_1_0_1,pred_1_0_0 = generate_sequence(X_start_A,seq_length)
sequence

<tf.Tensor: shape=(1, 40, 31), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.]]], dtype=float32)>

In [10]:
sequence = tf.reshape(sequence,[20+2*seq_length,31])
sequence = np.array(sequence)
sequence.shape

(40, 31)

In [11]:
cantus_lst = pred_1_0_0;
counter_lst = pred_0_1_1;
pred_cantus_lst = pred_0_1_0
pred_counter_lst = pred_1_0_1

In [12]:
cantus_lst

[57, 55, 62, 62, 59, 60, 65, 58, 52, 56]

In [13]:
pred_cantus_lst

[62, 61, 66, 58, 66, 64, 62, 56, 60, 51]

In [14]:
counter_lst

[70, 65, 69, 65, 73, 72, 66, 65, 68, 58]

In [17]:
pred_counter_lst

[60, 63, 70, 71, 62, 69, 69, 67, 61, 60]

In [18]:
np.array(counter_lst) - np.array(cantus_lst) # real harmonic

array([13, 10,  7,  3, 14, 12,  1,  7, 16,  2])

In [19]:
np.array(counter_lst) - np.array(pred_cantus_lst) # 0_1_harmonic

array([8, 4, 3, 7, 7, 8, 4, 9, 8, 7])

In [20]:
np.array(pred_counter_lst) - np.array(cantus_lst) # 1_0_harmonic

array([3, 8, 8, 9, 3, 9, 4, 9, 9, 4])

In [21]:
np.diff(cantus_lst)

array([-2,  7,  0, -3,  1,  5, -7, -6,  4])

In [116]:
np.diff(pred_cantus_lst)

array([-3,  4, -1,  0, -7,  0, -6,  6, -1])

In [117]:
np.diff(counter_lst)

array([ 0,  4, -5,  0, -6, -1, -5,  6, -5])

In [118]:
np.diff(pred_counter_lst)

array([  3,   9, -10,  -5,   1,  -3,   9,  -9,   0])

In [119]:
np.array(cantus_lst) - np.array(pred_cantus_lst)

array([-4,  1,  1, -7, -7, -4, -7,  8, -3, -1])

In [120]:
np.array(counter_lst) - np.array(pred_counter_lst)

array([  5,   2,  -3,   2,   7,   0,   2, -12,   3,  -2])

In [22]:
s = Session()
s.tempo = 180
piano1 = s.new_part("piano")
piano2 = s.new_part("piano")

def cantus():
    for i in cantus_lst:
        piano2.play_note(i,1,4)
        
def counter():
    for i in counter_lst:
        piano1.play_note(i,1,4)
        
        
s.fast_forward_to_beat(100)

s.start_transcribing()
s.fork(counter)
s.fork(cantus)
s.wait_for_children_to_finish()
performance = s.stop_transcribing()
performance.to_score(title = "First Species Counterpoint", composer = "My programme",time_signature = "4/4").show_xml()

Using preset Piano Merlin for piano
Using preset Piano Merlin for piano


In [26]:
X

<tf.Tensor: shape=(20000, 31), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [27]:
tf.random.shuffle([[1, 2],
 [3, 4], 
 [5, 6]])

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[3, 4],
       [5, 6],
       [1, 2]])>

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices(X[:9600,:]) # !!!!
test_dataset = tf.data.Dataset.from_tensor_slices(X[9600:,:])

In [10]:
n_steps = 20
window_length = n_steps
dataset = train_dataset.window(window_length, shift=2, drop_remainder=True)

In [11]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [14]:
batch_size = 20
shuffle_dataset = dataset.shuffle(10000).batch(batch_size,drop_remainder=True)

In [15]:
i = 0
for window in shuffle_dataset:
    i = i + 1
    print(window.shape)
    #print(window)

print(i)

(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)
(20, 20, 31)

In [13]:
# one step forward
# without communication
# without dominate - subordinate relation

counter = 0;
for window in shuffle_dataset: #(18,20,31)
    
    output_A_0_1_ = sub_model_0_1_.predict(window) #(18,30)
    out_0_1_0 = np.zeros([batch_size,1,31]) #(18,1,31)
    for i in range(batch_size):
        pred_A_class_random_0_1_ = np.random.choice(30, 1, p=output_A_0_1_[i,:])[0]
        one_hot = tf.reshape(tf.one_hot(pred_A_class_random_0_1_, 30),[1,1,30])
        out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)
    
    output_RNN, y_pred_A_0_1_, y_pred_B_0_1_ = model_0_1_.predict((window,out_0_1_0))
    y_pred_B_0_1_  #(18,30)
    out_0_1_1 = np.zeros([batch_size,1,31]) #(18,1,31)
    for i in range(batch_size):
        pred_B_class_random_0_1_ = np.random.choice(30, 1, p=y_pred_B_0_1_[i,:])[0]
        one_hot = tf.reshape(tf.one_hot(pred_B_class_random_0_1_, 30),[1,1,30])
        out_0_1_1[i:i+1,:,:]= np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)
    
    output_A_1_0_ = sub_model_1_0_.predict(window) #(18,30)
    out_1_0_1 = np.zeros([batch_size,1,31]) #(18,1,31)
    for i in range(batch_size):
        pred_A_class_random_1_0_ = np.random.choice(30, 1, p=output_A_1_0_[i,:])[0]
        one_hot = tf.reshape(tf.one_hot(pred_A_class_random_1_0_, 30),[1,1,30])
        out_1_0_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)
        
    output_RNN, y_pred_A_1_0_, y_pred_B_1_0_ = model_1_0_.predict((window,out_1_0_1))
    y_pred_B_1_0_  #(18,30)
    out_1_0_0 = np.zeros([batch_size,1,31]) #(18,1,31)
    for i in range(batch_size):
        pred_B_class_random_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
        one_hot = tf.reshape(tf.one_hot(pred_B_class_random_1_0_, 30),[1,1,30])
        out_1_0_0[i:i+1,:,:]= np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)
        
    target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1]
    target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1]
    target_1_0_1 = np.reshape(out_1_0_1,[batch_size,31])[:,:-1]
    target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1]
    
    if counter%40 == 0:
        sub_model_0_1_.fit(initial,target_1_0_0,verbose = 1)
        sub_model_1_0_.fit(initial,target_0_1_1,verbose = 1)
    else:
        sub_model_0_1_.fit(initial,target_1_0_0,verbose = 0)
        sub_model_1_0_.fit(initial,target_0_1_1,verbose = 0)
        
    counter = counter + 1

1/1 [==============================] - 0s 0s/step - loss: 2.9324


In [14]:
# n steps forward
# without communication
# without dominate - subordinate relation
n = 3
counter = 0;
for window in shuffle_dataset: #(18,20,31)
    initial = window
    
    for i in range(n): # n step forward

        output_A_0_1_ = sub_model_0_1_.predict(initial) #(18,30)
        out_0_1_0 = np.zeros([batch_size,1,31]) #(18,1,31)
        for i in range(batch_size):
            pred_A_class_random_0_1_ = np.random.choice(30, 1, p=output_A_0_1_[i,:])[0]
            one_hot = tf.reshape(tf.one_hot(pred_A_class_random_0_1_, 30),[1,1,30])
            out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

        output_RNN, y_pred_A_0_1_, y_pred_B_0_1_ = model_0_1_.predict((initial,out_0_1_0))
        y_pred_B_0_1_  #(18,30)
        out_0_1_1 = np.zeros([batch_size,1,31]) #(18,1,31)
        for i in range(batch_size):
            pred_B_class_random_0_1_ = np.random.choice(30, 1, p=y_pred_B_0_1_[i,:])[0]
            one_hot = tf.reshape(tf.one_hot(pred_B_class_random_0_1_, 30),[1,1,30])
            out_0_1_1[i:i+1,:,:]= np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

        output_A_1_0_ = sub_model_1_0_.predict(initial) #(18,30)
        out_1_0_1 = np.zeros([batch_size,1,31]) #(18,1,31)
        for i in range(batch_size):
            pred_A_class_random_1_0_ = np.random.choice(30, 1, p=output_A_1_0_[i,:])[0]
            one_hot = tf.reshape(tf.one_hot(pred_A_class_random_1_0_, 30),[1,1,30])
            out_1_0_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

        output_RNN, y_pred_A_1_0_, y_pred_B_1_0_ = model_1_0_.predict((initial,out_1_0_1))
        y_pred_B_1_0_  #(18,30)
        out_1_0_0 = np.zeros([batch_size,1,31]) #(18,1,31)
        for i in range(batch_size):
            pred_B_class_random_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
            one_hot = tf.reshape(tf.one_hot(pred_B_class_random_1_0_, 30),[1,1,30])
            out_1_0_0[i:i+1,:,:]= np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

        target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1]
        target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1]
        target_1_0_1 = np.reshape(out_1_0_1,[batch_size,31])[:,:-1]
        target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1]

        if counter%(n*40) == 0:
            sub_model_0_1_.fit(initial,target_1_0_0,verbose = 1)
            sub_model_1_0_.fit(initial,target_0_1_1,verbose = 1)
        else:
            sub_model_0_1_.fit(initial,target_1_0_0,verbose = 0)
            sub_model_1_0_.fit(initial,target_0_1_1,verbose = 0)
            
        counter = counter + 1
        initial = tf.concat((initial[:,:-2,:],out_1_0_0,out_0_1_1),1)

1/1 [==============================] - 0s 997us/step - loss: 2.2405


In [127]:
# version 1
# 1 - n steps forward
# without communication
# without dominate - subordinate relation

step = 15
training_times = 3;
counter = 0;
loss_0_1_ = 0
loss_1_0_ = 0
for n in range(1,step+1):
    for times in range(training_times):
        for window in shuffle_dataset: #(18,20,31)
            initial = window
            
            for i in range(n): # n step forward
                counter = counter + 1

                output_A_0_1_ = sub_model_0_1_.predict(initial) #(18,30)
                out_0_1_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_A_class_random_0_1_ = np.random.choice(30, 1, p=output_A_0_1_[i,:])[0]
                    pred_A_class_max_0_1_ = np.where(output_A_0_1_[i,:] == np.max(output_A_0_1_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_0_1_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_A_class_max_0_1_, 30),[1,1,30])

                    out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

                output_RNN, y_pred_A_0_1_, y_pred_B_0_1_ = model_0_1_.predict((initial,out_0_1_0))
                y_pred_B_0_1_  #(18,30)
                out_0_1_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_B_class_random_0_1_ = np.random.choice(30, 1, p=y_pred_B_0_1_[i,:])[0]
                    pred_B_class_max_0_1_ = np.where(y_pred_B_0_1_[i,:] == np.max(y_pred_B_0_1_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_0_1_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_B_class_max_0_1_, 30),[1,1,30])

                    out_0_1_1[i:i+1,:,:]= np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                output_A_1_0_ = sub_model_1_0_.predict(initial) #(18,30)
                out_1_0_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_A_class_random_1_0_ = np.random.choice(30, 1, p=output_A_1_0_[i,:])[0]
                    pred_A_class_max_1_0_ = np.where(output_A_1_0_[i,:] == np.max(output_A_1_0_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_1_0_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_A_class_max_1_0_, 30),[1,1,30])

                    out_1_0_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                output_RNN, y_pred_A_1_0_, y_pred_B_1_0_ = model_1_0_.predict((initial,out_1_0_1))
                y_pred_B_1_0_  #(18,30)
                out_1_0_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_B_class_random_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
                    pred_B_class_max_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_1_0_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_B_class_max_1_0_, 30),[1,1,30])

                    out_1_0_0[i:i+1,:,:]= np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

                target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1]
                target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1]
                target_1_0_1 = np.reshape(out_1_0_1,[batch_size,31])[:,:-1]
                target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1]

                
                History_0_1_ = sub_model_0_1_.fit(initial,target_1_0_0,verbose = 0)
                loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]

                History_1_0_ = sub_model_1_0_.fit(initial,target_0_1_1,verbose = 0)
                loss_1_0_ = loss_1_0_ + History_1_0_.history.get('loss')[0]
                
                if counter%1000 == 0:
                    
                    print('model_0_1 - loss: ', loss_0_1_/1000)
                    print('model_1_0 - loss: ', loss_1_0_/1000)
                    loss_0_1_ = 0
                    loss_1_0_ = 0

                initial = tf.concat((initial[:,:-2,:],out_1_0_0,out_0_1_1),1)

model_0_1 - loss:  0.2652583754900843
model_1_0 - loss:  0.3246574815874919
model_0_1 - loss:  0.006557741165626794
model_1_0 - loss:  0.006984730981756002
model_0_1 - loss:  0.0036032973157707603
model_1_0 - loss:  0.003756456454982981
model_0_1 - loss:  0.0024886268044356256
model_1_0 - loss:  0.0025877053467556832
model_0_1 - loss:  0.0018996830940013751
model_1_0 - loss:  0.0019765501485671847
model_0_1 - loss:  0.0015337823668960483
model_1_0 - loss:  0.0015962444654433058
model_0_1 - loss:  0.0012892459457507356
model_1_0 - loss:  0.0013434799746610223
model_0_1 - loss:  0.0011068339052144438
model_1_0 - loss:  0.0011554455670993776
model_0_1 - loss:  0.0009724555585416965
model_1_0 - loss:  0.0010175566663965584
model_0_1 - loss:  0.0008679081986192614
model_1_0 - loss:  0.0009080864900024608
model_0_1 - loss:  0.0007805454772314988
model_1_0 - loss:  0.0008171391857322305
model_0_1 - loss:  0.0007118273568921722
model_1_0 - loss:  0.0007459093842771835
model_0_1 - loss:  0.0006

KeyboardInterrupt: 

In [28]:
History.history.get('loss')[0]

2.6011555194854736

In [79]:
# version 2
# 1 - n steps forward
# with communication
# without dominate - subordinate relation
step = 15
training_times = 3;
counter = 0;
loss_0_1_ = 0
loss_1_0_ = 0
for n in range(1,step+1):
    for times in range(training_times):
        for window in shuffle_dataset: #(18,20,31)
            initial = window

            for i in range(n): # n step forward
                counter = counter + 1

                output_A_0_1_ = sub_model_0_1_.predict(initial) #(18,30)
                out_0_1_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_A_class_random_0_1_ = np.random.choice(30, 1, p=output_A_0_1_[i,:])[0]
                    pred_A_class_max_0_1_ = np.where(output_A_0_1_[i,:] == np.max(output_A_0_1_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_0_1_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_A_class_max_0_1_, 30),[1,1,30])

                    out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

                output_RNN, y_pred_A_0_1_, y_pred_B_0_1_ = model_0_1_.predict((initial,out_0_1_0))
                y_pred_B_0_1_  #(18,30)
                out_0_1_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_B_class_random_0_1_ = np.random.choice(30, 1, p=y_pred_B_0_1_[i,:])[0]
                    pred_B_class_max_0_1_ = np.where(y_pred_B_0_1_[i,:] == np.max(y_pred_B_0_1_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_0_1_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_B_class_max_0_1_, 30),[1,1,30])

                    out_0_1_1[i:i+1,:,:]= np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                output_A_1_0_ = sub_model_1_0_.predict(initial) #(18,30)
                out_1_0_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_A_class_random_1_0_ = np.random.choice(30, 1, p=output_A_1_0_[i,:])[0]
                    pred_A_class_max_1_0_ = np.where(output_A_1_0_[i,:] == np.max(output_A_1_0_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_1_0_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_A_class_max_1_0_, 30),[1,1,30])

                    out_1_0_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                output_RNN, y_pred_A_1_0_, y_pred_B_1_0_ = model_1_0_.predict((initial,out_1_0_1))
                y_pred_B_1_0_  #(18,30)
                out_1_0_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_B_class_random_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
                    pred_B_class_max_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_1_0_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_B_class_max_1_0_, 30),[1,1,30])

                    out_1_0_0[i:i+1,:,:]= np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)


                target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1]
                target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1]
                target_1_0_1 = np.reshape(out_1_0_1,[batch_size,31])[:,:-1]
                target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1]

                History_0_1_ = aux_model_0_1_.fit([initial,out_1_0_0],[target_1_0_0,target_1_0_1],verbose = 0)
                loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]

                History_1_0_ = aux_model_1_0_.fit([initial,out_0_1_1],[target_0_1_1,target_0_1_0],verbose = 0)
                loss_1_0_ = loss_1_0_ + History_1_0_.history.get('loss')[0]
                
                if counter%1000 == 0:
                    
                    print('model_0_1 - loss: ', loss_0_1_/1000)
                    print('model_1_0 - loss: ', loss_1_0_/1000)
                    loss_0_1_ = 0
                    loss_1_0_ = 0

                
                initial = tf.concat((initial[:,:-2,:],out_1_0_0,out_0_1_1),1)

model_0_1 - loss:  0.20620975234825165
model_1_0 - loss:  0.22923851236514747
model_0_1 - loss:  0.01070334950229153
model_1_0 - loss:  0.011377248265780508
model_0_1 - loss:  0.0062285731215961275
model_1_0 - loss:  0.0064736071899533275
model_0_1 - loss:  0.004412614834029227
model_1_0 - loss:  0.004532194695202634
model_0_1 - loss:  0.003423396444413811
model_1_0 - loss:  0.0034866099236533048
model_0_1 - loss:  0.0027973438538610937
model_1_0 - loss:  0.002828990068985149
model_0_1 - loss:  0.002366253442829475
model_1_0 - loss:  0.0023807444309350105
model_0_1 - loss:  0.0020480897487141192
model_1_0 - loss:  0.0020524442527676003
model_0_1 - loss:  0.0018081698580645026
model_1_0 - loss:  0.0018049650323810056
model_0_1 - loss:  0.0016171624665148557
model_1_0 - loss:  0.0016097894302802161
model_0_1 - loss:  0.0014629098530858755
model_1_0 - loss:  0.0014516551727429031
model_0_1 - loss:  0.0013364156092284247
model_1_0 - loss:  0.0013231156929396093
model_0_1 - loss:  0.0012296

KeyboardInterrupt: 

In [102]:
# version 3
# 1 - n steps forward
# without communication
# with single side dominate - subordinate relation

# same as:
# adapt my mind to the observation
# no action applied

step = 15
training_times = 3;
counter = 0;
loss_0_1_ = 0

for n in range(1,step+1):
    for times in range(training_times):
        for window in shuffle_dataset: #(18,20,31)
            initial = window

            for i in range(n): # n step forward
                counter = counter + 1

                output_A_0_1_ = sub_model_0_1_.predict(initial) #(18,30)
                out_0_1_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_A_class_random_0_1_ = np.random.choice(30, 1, p=output_A_0_1_[i,:])[0]
                    pred_A_class_max_0_1_ = np.where(output_A_0_1_[i,:] == np.max(output_A_0_1_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_0_1_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_A_class_max_0_1_, 30),[1,1,30])

                    out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

                output_RNN, y_pred_A_0_1_, y_pred_B_0_1_ = model_0_1_.predict((initial,out_0_1_0))
                y_pred_B_0_1_  #(18,30)
                out_0_1_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_B_class_random_0_1_ = np.random.choice(30, 1, p=y_pred_B_0_1_[i,:])[0]
                    pred_B_class_max_0_1_ = np.where(y_pred_B_0_1_[i,:] == np.max(y_pred_B_0_1_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_0_1_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_B_class_max_0_1_, 30),[1,1,30])

                    out_0_1_1[i:i+1,:,:]= np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                output_A_1_0_ = sub_model_1_0_.predict(initial) #(18,30)
                out_1_0_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_A_class_random_1_0_ = np.random.choice(30, 1, p=output_A_1_0_[i,:])[0]
                    pred_A_class_max_1_0_ = np.where(output_A_1_0_[i,:] == np.max(output_A_1_0_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_1_0_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_A_class_max_1_0_, 30),[1,1,30])

                    out_1_0_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                output_RNN, y_pred_A_1_0_, y_pred_B_1_0_ = model_1_0_.predict((initial,out_1_0_1))
                y_pred_B_1_0_  #(18,30)
                out_1_0_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_B_class_random_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
                    pred_B_class_max_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_1_0_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_B_class_max_1_0_, 30),[1,1,30])

                    out_1_0_0[i:i+1,:,:]= np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)


                target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1]
                target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1]
                target_1_0_1 = np.reshape(out_1_0_1,[batch_size,31])[:,:-1]
                target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1]

                History_0_1_ = sub_model_0_1_.fit(initial,target_1_0_0,verbose = 0)
                loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]
                
                if counter%1000 == 0:
                    print('model_0_1 - loss: ', loss_0_1_/1000)
                    loss_0_1_ = 0
                    
                initial = tf.concat((initial[:,:-2,:],out_1_0_0,out_0_1_1),1)

model_0_1 - loss:  1.9853931000232696
model_0_1 - loss:  1.763319775223732
model_0_1 - loss:  1.6686825339794158
model_0_1 - loss:  1.5990980682969094
model_0_1 - loss:  1.5432597593069077
model_0_1 - loss:  1.5040169832110406
model_0_1 - loss:  1.4893821044564246
model_0_1 - loss:  1.4353866910338402
model_0_1 - loss:  1.3946545179486274
model_0_1 - loss:  1.3794402884840966
model_0_1 - loss:  1.3635864284634591
model_0_1 - loss:  1.3378718311190605
model_0_1 - loss:  1.296607871234417
model_0_1 - loss:  1.315218718290329
model_0_1 - loss:  1.2583481413125992
model_0_1 - loss:  1.227064264535904
model_0_1 - loss:  1.2437001282572746
model_0_1 - loss:  1.2070959852337837
model_0_1 - loss:  1.19810886323452
model_0_1 - loss:  1.1733711369037627
model_0_1 - loss:  1.1736106811761855
model_0_1 - loss:  1.1214050613045692
model_0_1 - loss:  1.1285535547733307
model_0_1 - loss:  1.112572342157364
model_0_1 - loss:  1.1029101669788361
model_0_1 - loss:  1.1022457077503205
model_0_1 - loss:  

In [141]:
# version 4
# 1 - n steps forward
# without communication
# with either side dominate - subordinate relation
step = 15
training_times = 3;
counter = 0;
loss_0_1_ = 0
loss_1_0_ = 0
for n in range(1,step+1):
    for times in range(training_times):
        for window in shuffle_dataset: #(18,20,31)
            initial = window

            for i in range(n): # n step forward
                counter = counter + 1

                output_A_0_1_ = sub_model_0_1_.predict(initial) #(18,30)
                out_0_1_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_A_class_random_0_1_ = np.random.choice(30, 1, p=output_A_0_1_[i,:])[0]
                    pred_A_class_max_0_1_ = np.where(output_A_0_1_[i,:] == np.max(output_A_0_1_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_0_1_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_A_class_max_0_1_, 30),[1,1,30])

                    out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

                output_RNN, y_pred_A_0_1_, y_pred_B_0_1_ = model_0_1_.predict((initial,out_0_1_0))
                y_pred_B_0_1_  #(18,30)
                out_0_1_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_B_class_random_0_1_ = np.random.choice(30, 1, p=y_pred_B_0_1_[i,:])[0]
                    pred_B_class_max_0_1_ = np.where(y_pred_B_0_1_[i,:] == np.max(y_pred_B_0_1_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_0_1_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_B_class_max_0_1_, 30),[1,1,30])

                    out_0_1_1[i:i+1,:,:]= np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                output_A_1_0_ = sub_model_1_0_.predict(initial) #(18,30)
                out_1_0_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_A_class_random_1_0_ = np.random.choice(30, 1, p=output_A_1_0_[i,:])[0]
                    pred_A_class_max_1_0_ = np.where(output_A_1_0_[i,:] == np.max(output_A_1_0_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_1_0_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_A_class_max_1_0_, 30),[1,1,30])

                    out_1_0_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                output_RNN, y_pred_A_1_0_, y_pred_B_1_0_ = model_1_0_.predict((initial,out_1_0_1))
                y_pred_B_1_0_  #(18,30)
                out_1_0_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_B_class_random_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
                    pred_B_class_max_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_1_0_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_B_class_max_1_0_, 30),[1,1,30])

                    out_1_0_0[i:i+1,:,:]= np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)


                target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1]
                target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1]
                target_1_0_1 = np.reshape(out_1_0_1,[batch_size,31])[:,:-1]
                target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1]

                if np.random.rand(1)[0] < 0.5:
                    History_0_1_ = sub_model_0_1_.fit(initial,target_1_0_0,verbose = 0) 
                    loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]
                else:
                    History_1_0_ = sub_model_1_0_.fit(initial,target_0_1_1,verbose = 0)
                    loss_1_0_ = loss_1_0_ + History_1_0_.history.get('loss')[0]

                if counter%2000 == 0:
                    
                    print('model_0_1 - loss: ', loss_0_1_/1000)
                    print('model_1_0 - loss: ', loss_1_0_/1000)
                    loss_0_1_ = 0
                    loss_1_0_ = 0
                    
                initial = tf.concat((initial[:,:-2,:],out_1_0_0,out_0_1_1),1)

model_0_1 - loss:  0.43955828779470174
model_1_0 - loss:  0.4716214130232111
model_0_1 - loss:  0.007909399166703223
model_1_0 - loss:  0.007167036981787532
model_0_1 - loss:  0.003696445174748078
model_1_0 - loss:  0.003912661943119019
model_0_1 - loss:  0.0025852611139416696
model_1_0 - loss:  0.0025333449493627996
model_0_1 - loss:  0.0019399746045237407
model_1_0 - loss:  0.0019240923252655192
model_0_1 - loss:  0.0015254788940073922
model_1_0 - loss:  0.0015837491110432893
model_0_1 - loss:  0.0012355579438153655
model_1_0 - loss:  0.0013610715855611488
model_0_1 - loss:  0.001060215443954803
model_1_0 - loss:  0.0011669691242277623
model_0_1 - loss:  0.000959481451427564
model_1_0 - loss:  0.0009942673149635085
model_0_1 - loss:  0.000831947589002084
model_1_0 - loss:  0.0009055787406978198
model_0_1 - loss:  0.0007486671467195265
model_1_0 - loss:  0.0008168068425729871
model_0_1 - loss:  0.0007050269345054402
model_1_0 - loss:  0.0007186501563992352
model_0_1 - loss:  0.0006484

In [55]:
# version 5
# 1 - n steps forward
# with communication
# with single side dominate - subordinate relation

step = 15
training_times = 3;
counter = 0;
loss_0_1_ = 0;
for epoch in range(1):
    for n in range(1,step+1):
        for times in range(training_times):
            for window in shuffle_dataset: #(18,20,31)
                initial = window

                for i in range(n): # n step forward
                    counter = counter + 1

                    output_A_0_1_ = sub_model_0_1_.predict(initial) #(18,30)
                    out_0_1_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                    for i in range(batch_size):
                        # pred_A_class_random_0_1_ = np.random.choice(30, 1, p=output_A_0_1_[i,:])[0]
                        pred_A_class_max_0_1_ = np.where(output_A_0_1_[i,:] == np.max(output_A_0_1_[i,:]))[0][0]
                        
                        # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_0_1_, 30),[1,1,30])
                        one_hot = tf.reshape(tf.one_hot(pred_A_class_max_0_1_, 30),[1,1,30])
                        
                        out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

                    output_RNN, y_pred_A_0_1_, y_pred_B_0_1_ = model_0_1_.predict((initial,out_0_1_0))
                    y_pred_B_0_1_  #(18,30)
                    out_0_1_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                    for i in range(batch_size):
                        # pred_B_class_random_0_1_ = np.random.choice(30, 1, p=y_pred_B_0_1_[i,:])[0]
                        pred_B_class_max_0_1_ = np.where(y_pred_B_0_1_[i,:] == np.max(y_pred_B_0_1_[i,:]))[0][0]
                        
                        # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_0_1_, 30),[1,1,30])
                        one_hot = tf.reshape(tf.one_hot(pred_B_class_max_0_1_, 30),[1,1,30])
                        
                        out_0_1_1[i:i+1,:,:]= np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                    output_A_1_0_ = sub_model_1_0_.predict(initial) #(18,30)
                    out_1_0_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                    for i in range(batch_size):
                        # pred_A_class_random_1_0_ = np.random.choice(30, 1, p=output_A_1_0_[i,:])[0]
                        pred_A_class_max_1_0_ = np.where(output_A_1_0_[i,:] == np.max(output_A_1_0_[i,:]))[0][0]
                        
                        # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_1_0_, 30),[1,1,30])
                        one_hot = tf.reshape(tf.one_hot(pred_A_class_max_1_0_, 30),[1,1,30])
                        
                        out_1_0_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                    output_RNN, y_pred_A_1_0_, y_pred_B_1_0_ = model_1_0_.predict((initial,out_1_0_1))
                    y_pred_B_1_0_  #(18,30)
                    out_1_0_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                    for i in range(batch_size):
                        # pred_B_class_random_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
                        pred_B_class_max_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]
                        
                        # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_1_0_, 30),[1,1,30])
                        one_hot = tf.reshape(tf.one_hot(pred_B_class_max_1_0_, 30),[1,1,30])
                        
                        out_1_0_0[i:i+1,:,:]= np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

                    target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1]
                    target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1]
                    target_1_0_1 = np.reshape(out_1_0_1,[batch_size,31])[:,:-1]
                    target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1]

                    History_0_1_ = aux_model_0_1_.fit([initial,out_1_0_0],[target_1_0_0,target_1_0_1],verbose = 0)
                    loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]

                    if counter%1000 == 0:
                        print('model_0_1 - loss: ', loss_0_1_/1000)
                        loss_0_1_ = 0

                    initial = tf.concat((initial[:,:-2,:],out_1_0_0,out_0_1_1),1)

model_0_1 - loss:  2.734954721689224
model_0_1 - loss:  2.2768907078504563
model_0_1 - loss:  2.07823702788353
model_0_1 - loss:  1.9759793725013732
model_0_1 - loss:  1.8553427128791808
model_0_1 - loss:  1.7616099138855934
model_0_1 - loss:  1.7434243019223212
model_0_1 - loss:  1.6472704045772553
model_0_1 - loss:  1.5884761747717857
model_0_1 - loss:  1.6012948861718177
model_0_1 - loss:  1.5375391768217086
model_0_1 - loss:  1.4962132036685944
model_0_1 - loss:  1.459306826353073
model_0_1 - loss:  1.4436699231266976
model_0_1 - loss:  1.4214068812131881
model_0_1 - loss:  1.38073533898592
model_0_1 - loss:  1.389890702188015
model_0_1 - loss:  1.3308183543086052
model_0_1 - loss:  1.3199413604140282
model_0_1 - loss:  1.3131540709733962
model_0_1 - loss:  1.286356808781624
model_0_1 - loss:  1.2619687846899033
model_0_1 - loss:  1.2399959751367569
model_0_1 - loss:  1.244893409192562
model_0_1 - loss:  1.2210461108088493
model_0_1 - loss:  1.247340129494667
model_0_1 - loss:  1.1

In [16]:
# version 6
# 1 - n steps forward
# with communication
# with either side dominate - subordinate relation
step = 15
training_times = 3;
counter = 0;
loss_0_1_ = 0
loss_1_0_ = 0
for epoch in range(1):
    for n in range(1,step+1):
        for times in range(training_times):
            for window in shuffle_dataset: #(18,20,31)
                initial = window

                for i in range(n): # n step forward
                    counter = counter + 1

                    output_A_0_1_ = sub_model_0_1_.predict(initial) #(18,30)
                    out_0_1_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                    for i in range(batch_size):
                        # pred_A_class_random_0_1_ = np.random.choice(30, 1, p=output_A_0_1_[i,:])[0]
                        pred_A_class_max_0_1_ = np.where(output_A_0_1_[i,:] == np.max(output_A_0_1_[i,:]))[0][0]
                        
                        # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_0_1_, 30),[1,1,30])
                        one_hot = tf.reshape(tf.one_hot(pred_A_class_max_0_1_, 30),[1,1,30])
                        
                        out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

                    output_RNN, y_pred_A_0_1_, y_pred_B_0_1_ = model_0_1_.predict((initial,out_0_1_0))
                    y_pred_B_0_1_  #(18,30)
                    out_0_1_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                    for i in range(batch_size):
                        # pred_B_class_random_0_1_ = np.random.choice(30, 1, p=y_pred_B_0_1_[i,:])[0]
                        pred_B_class_max_0_1_ = np.where(y_pred_B_0_1_[i,:] == np.max(y_pred_B_0_1_[i,:]))[0][0]
                        
                        # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_0_1_, 30),[1,1,30])
                        one_hot = tf.reshape(tf.one_hot(pred_B_class_max_0_1_, 30),[1,1,30])
                        
                        out_0_1_1[i:i+1,:,:]= np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                    output_A_1_0_ = sub_model_1_0_.predict(initial) #(18,30)
                    out_1_0_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                    for i in range(batch_size):
                        # pred_A_class_random_1_0_ = np.random.choice(30, 1, p=output_A_1_0_[i,:])[0]
                        pred_A_class_max_1_0_ = np.where(output_A_1_0_[i,:] == np.max(output_A_1_0_[i,:]))[0][0]
                        
                        # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_1_0_, 30),[1,1,30])
                        one_hot = tf.reshape(tf.one_hot(pred_A_class_max_1_0_, 30),[1,1,30])
                        
                        out_1_0_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                    output_RNN, y_pred_A_1_0_, y_pred_B_1_0_ = model_1_0_.predict((initial,out_1_0_1))
                    y_pred_B_1_0_  #(18,30)
                    out_1_0_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                    for i in range(batch_size):
                        # pred_B_class_random_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
                        pred_B_class_max_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]
                        
                        # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_1_0_, 30),[1,1,30])
                        one_hot = tf.reshape(tf.one_hot(pred_B_class_max_1_0_, 30),[1,1,30])
                        
                        out_1_0_0[i:i+1,:,:]= np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

                    target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1]
                    target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1]
                    target_1_0_1 = np.reshape(out_1_0_1,[batch_size,31])[:,:-1]
                    target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1]

                    if np.random.rand(1)[0] < 0.5:
                        History_0_1_ = aux_model_0_1_.fit([initial,out_1_0_0],[target_1_0_0,target_1_0_1],verbose = 0)
                        loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]
                    else:
                        History_1_0_ = aux_model_1_0_.fit([initial,out_0_1_1],[target_0_1_1,target_0_1_0],verbose = 0)
                        loss_1_0_ = loss_1_0_ + History_1_0_.history.get('loss')[0]

                    if counter%2000 == 0:

                        print('model_0_1 - loss: ', loss_0_1_/1000)
                        print('model_1_0 - loss: ', loss_1_0_/1000)
                        loss_0_1_ = 0
                        loss_1_0_ = 0

                    initial = tf.concat((initial[:,:-2,:],out_1_0_0,out_0_1_1),1)

model_0_1 - loss:  0.1747257892070338
model_1_0 - loss:  0.18830171506665647
model_0_1 - loss:  0.010841891036834568
model_1_0 - loss:  0.010778524483554066
model_0_1 - loss:  0.006334565908182412
model_1_0 - loss:  0.006184166882652789
model_0_1 - loss:  0.00428621018002741
model_1_0 - loss:  0.004562880933983251
model_0_1 - loss:  0.0034175378396175803
model_1_0 - loss:  0.003422798012616113
model_0_1 - loss:  0.0027646566529292613
model_1_0 - loss:  0.002819224027916789
model_0_1 - loss:  0.0023918168831150977
model_1_0 - loss:  0.002316170803969726
model_0_1 - loss:  0.0019415372186340392
model_1_0 - loss:  0.002127920774393715
model_0_1 - loss:  0.0017300101198488846
model_1_0 - loss:  0.0018551852572709323
model_0_1 - loss:  0.0016110781442839651
model_1_0 - loss:  0.001594420215464197
model_0_1 - loss:  0.0015399826674256474
model_1_0 - loss:  0.0013579110694117843
model_0_1 - loss:  0.0013480629787081852
model_1_0 - loss:  0.001292885557981208
model_0_1 - loss:  0.0012365215381

In [142]:
model_0_1_.save_weights('version4_withoutCommunication_withEitherDomSub_model_0_1_.h5')
model_1_0_.save_weights('version4_withoutCommunication_withEitherDomSub_model_1_0_.h5')

In [37]:
prob = np.zeros((30,))
prob

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [40]:
#prob = np.array([1,4,5])
np.where(prob == np.max(prob))[0][0]

0

In [52]:
out_1_0_1 =  tf.reshape(prob,[2,1,15])
one = tf.reshape(tf.ones(2),[2,1,1])
out_1_0_1 =  np.concatenate((out_1_0_1,one),2) #(18,1,31)
out_1_0_1

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]])

In [49]:
out_1_0_1

<tf.Tensor: shape=(2, 1, 15), dtype=float64, numpy=
array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])>

In [51]:
tf.ones(2)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>